# Compare model outputs of AICORE DL Model
#### Tasks
- detect resumed config

```
model:
  architecture: Unet
  encoder: resnet50
  encoder_weights: random
  input_channels: 10
model_args:
  norm: BatchNorm
  output_channels: 2
  stack_height: 6
resume: !!python/object/apply:pathlib.PosixPath
- logs
- CV6__V__Horton__UNet_resnet50
- checkpoints
- 50.pt
```

### Imports 

In [31]:
import plotly.express as px
import pandas as pd
import os
import glob
from pathlib import Path
import yaml

# workaround buggy autocomplete
%config Completer.use_jedi = False

### Functions 

In [73]:
def get_props_from_config(config_file):
    config = yaml.safe_load(open(config_file))
    m = config['model']
    return m['architecture'], m['encoder']

In [23]:
def get_vals_from_dir(subdir):
    basename = subdir.name
    
    df_tr = pd.read_csv(subdir / 'train.csv')
    df_tr['basename'] = basename[:]
    df_tr['type'] = 'train'
    df_list_tr.append(df_tr)
    
    
    df_v = pd.read_csv(s / 'val.csv')
    df_v['basename'] = basename[:]
    df_v['type'] = 'val'
    df_list_v.append(df_v)
    
    return df_tr, df_v

### Settings 

In [83]:
#LOGDIR = Path('logs') / '0.3.1'
LOGDIR = Path(r'O:\p_aicore\initze\aicore-uc2_augmentation\logs')
#SUB_REGEX = '*'
SUB_REGEX = '*CV6*'

In [84]:
subdirs = list(LOGDIR.glob(SUB_REGEX))

### Data Loading 
- add config reader

In [101]:
df_list_tr = []
df_list_v = []
for s in subdirs:
    try:
        df_tr, df_v = get_vals_from_dir(s)
    except:
        continue
    config_file = s / 'config.yml'
    try:
        architecture, backbone = get_props_from_config(config_file)
    except:
        architecture, backbone = '', ''
    df_tr['architecture'] = architecture
    df_tr['backbone'] = backbone
    df_tr['a-b'] = architecture + '_' + backbone
    df_v['architecture'] = architecture
    df_v['backbone'] = backbone
    df_v['a-b'] = architecture + '_' + backbone
    
    df_list_tr.append(df_tr)
    df_list_v.append(df_v)

df_train = pd.concat(df_list_tr).reset_index()
df_val = pd.concat(df_list_v).reset_index()

### Plots 

In [105]:
# put data into plottable shape
id_vars = ['Epoch', 'basename', 'type', 'architecture', 'a-b']

df_train_melt = df_train.melt(id_vars=id_vars, value_vars=['IoU']).sort_values(by='Epoch')
df_val_melt = df_val.melt(id_vars=id_vars, value_vars=['IoU']).sort_values(by='Epoch')
# merge
df_melt = pd.concat([df_val_melt])
#df_melt = pd.concat([df_train_melt, df_val_melt])

### Training data 

In [116]:
#px.line(df_melt, x='Epoch' , y='value' , color='basename', line_group='type')
p = px.line(df_train_melt, x='Epoch' , y='value' , color='basename', line_group='architecture')
p.update_layout(legend_orientation='h', )

#### Validation data 

In [117]:
#px.line(df_melt, x='Epoch' , y='value' , color='basename', line_group='type')
p = px.line(df_val_melt, x='Epoch' , y='value' , color='basename', line_group='architecture')
p.update_layout(legend_orientation='h', )

#### Plot mean values of configurations 

In [126]:
df_train_melt['ds'] = df_train_melt.apply(lambda x: x.basename.split('__')[0], axis=1)
mean_vals = df_train_melt.groupby(['Epoch', 'architecture']).mean().reset_index(drop=False)

In [127]:
px.line(mean_vals, x='Epoch', y='value', color='architecture')

In [128]:
df_val_melt['ds'] = df_val_melt.apply(lambda x: x.basename.split('__')[0], axis=1)
mean_vals = df_val_melt.groupby(['Epoch', 'architecture']).mean().reset_index(drop=False)

In [129]:
px.line(mean_vals, x='Epoch', y='value', color='architecture')